In [ ]:
import time
from pynq.overlays.base import BaseOverlay
import socket

base = BaseOverlay("base.bit")

# Sockets

This notebook has both a client and a server functionality. One PYNQ board in the group will be the client and SENDS the message. Another PYNQ board will be the server and RECEIVES the message.

## Server

Here, we'll build the server code to LISTEN for a message from a specific PYNQ board.

When we send/receive messages, we need to pieces of information which will tell us where to send the information. First, we need the IP address of our friend. Second, we need to chose a port to listen on. For an analogy, Alice expects her friend, Bob, to deliver a package to our back door. With this information, ALICE (server ip address) can wait at the BACK DOOR (port) for BOB (client ip address) to deliver the package.

Format of the information
 ipv4 address: ###.###.###.### (no need for leading zeros if the number is less than three digits)
 port: ##### (it could be 4 or 5 digits long, but must be >1024)
 
Use the socket documentation (Section 18.1.3) to find the appropriate functions https://python.readthedocs.io/en/latest/library/socket.html


In [24]:
HOST_ADDR = '192.168.0.102'
HOST_PORT = 65432
BUF_SIZE = 1024

def run_program():
    # TODO:
    # 1: Bind the socket to the pynq board <CLIENT-IP> at port <LISTENING-PORT>
    sock_l = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock_l.bind((HOST_ADDR, HOST_PORT))
    # 2: Accept connections
    sock_l.listen()
    print('Waiting for connection')
    conn, addr = sock_l.accept()
    print('Connected')
    # 3: Receive bytes from the connection
    with conn:
        data = conn.recv(BUF_SIZE)
        # 4: Print the received message
        print(data.decode())
    print("Connection closed.")


original_sigint = signal.getsignal(signal.SIGINT)
signal.signal(signal.SIGINT, exit)
run_program()

Waiting for connection
Connected
tets
Connection closed.


## Client

Now, we can implement the CLIENT code. 

Back to the analogy, now we're interested in delivering a package to our friend's back door. This means BOB (client ip address) is delivering a package to ALICE (server ip address) at her BACK DOOR (port)

**Remember to start the server before running the client code**

In [ ]:
HOST_ADDR = "192.168.0.100"  # ryan laptop
HOST_PORT = 65432

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    # TODO: 
    # 1: Connect the socket (sock) to the <SERVER-IP> and choosen port <LISTENING-PORT>
    sock.connect((HOST_ADDR, HOST_PORT))
    # 2: Send the message "Hello world!\n"
    sock.sendall(b"Hello world!\n")
    # 3: Close the socket
    # auto close

print("Connection closed.")

On your server, you should see the message and then the server will shutdown! When we close a socket, both the client and the server are disconnected from the port.

Instead, change the function above to send 5 messages before closing.

The pseudocode looks like this

* connect the socket
* for i in range(5)
    * msg = input("Message to send: ")
    * send the message (msg)
* close the socket

# Server with persistent socket

In [ ]:
import socket
import time
import signal
import sys

HOST_ADDR = '192.168.0.102'
HOST_PORT = 65432
BUF_SIZE = 1024

def run_program():
    sock_l = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock_l.bind((HOST_ADDR, HOST_PORT))
    sock_l.listen()
    print('Waiting for connection')
    conn, addr = sock_l.accept()
    print('Connected')
    with conn:
        for i in range(5):
            data = conn.recv(BUF_SIZE)
            print(data.decode())
    print("Connection closed.")


original_sigint = signal.getsignal(signal.SIGINT)
signal.signal(signal.SIGINT, exit)
run_program()

Waiting for connection
Connected
test
hello
thisisatest




# Client with persistent socket

In [20]:
HOST_ADDR = "192.168.0.100"  # ryan laptop
HOST_PORT = 65432

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    # TODO: 
    # 1: Connect the socket (sock) to the <SERVER-IP> and choosen port <LISTENING-PORT>
    sock.connect((HOST_ADDR, HOST_PORT))
    # 2: Send the message "Hello world!\n"
    for i in range(5):
        msg = input("Message to send: ")
        sock.sendall(str.encode(msg))
        print("Sent to server")
    # 3: Close the socket
    # auto close

print("Connection closed.")

Sent to server
Sent to server
Sent to server
Sent to server
Sent to server
Connection closed.
